In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Move into our project
os.chdir('/content/drive/MyDrive/crowd-counter-bot')

In [ ]:
# Use the old TF and downgrade h5py so we dont run into issues when loading our weights

%tensorflow_version 1.x
!pip install 'h5py==2.10.0' --force-reinstall
!pip install pillow

In [ ]:
from model.IUML import IUMLnet

# Load up the model
model = IUMLnet()
model.load_weights('model/IUML_best.hdf5')

In [ ]:
import cv2
import datetime
import requests
import numpy as np
from PIL import Image
from io import BytesIO


def request(url):
    """Predicts the number of people in an image given by the url.

    :param url: URL to the image
    :type urL: str

    :return: A tuple containing the number of people in the image and the execution time
    :rtype: tuple of floats
    """

    # Load up the image
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')

    start_time = datetime.datetime.now()

    # Convert from RGB to BGR
    img = np.array(img)
    img = img[:,:,::-1]

    # Network has been trained on 1024x768 dataset
    if img.shape != (768, 1024, 3):
        img = cv2.resize(img, (1024, 768))

    # Some required pre-processing
    img = img / 255.0
    img[:,:,0] = (img[:,:,0]-0.485) / 0.229
    img[:,:,1] = (img[:,:,1]-0.456) / 0.224
    img[:,:,2] = (img[:,:,2]-0.406) / 0.225

    # Require an input of shape (None, 768, 1024, 3)
    img = np.asarray([img])

    # Since we're not doing this in batches, we can just take the first index
    result = model.predict(img)[0]

    # Resize if needed
    if result.shape != (768, 1024, 1):
        result = cv2.resize(result, (1024, 768))

    end_time = datetime.datetime.now()

    time_diff = (end_time - start_time)
    execution_time = time_diff.total_seconds() * 1000   # in milliseconds

    return (np.sum(result), execution_time)

In [ ]:
# Startup a Discord bot to listen in on image requests

!pip install discord.py

# Ensure that our Jupyter notebook is able to handle async operations
!pip install nest_asyncio 
import nest_asyncio 
nest_asyncio.apply()

In [ ]:
import discord
import os
import requests

client = discord.Client()

@client.event
async def on_ready():
  print('We have logged in as {0.user}'.format(client))

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    # Predict only if there's an attachment
    if len(message.attachments) > 0:
        # Limit to the first attachment in case someone sends a stupidly big batch of images
        num_heads, execution_time = request(message.attachments[0].url)

        await message.channel.send(f"There are {num_heads:.2f} people in this image. It took {execution_time:.2f} ms to process.")
        return

API_KEY = 'YOUR_API_KEY_HERE'
client.run(API_KEY)